In [1]:
import requests
import json
import pandas as pd
import datetime

from utils import formatURL
from config import getConfig

ModuleNotFoundError: No module named 'pandas'

In [2]:
# TODO
# Where is the data for DIISX?

In [16]:
def getData(symbol, fromDt, toDt):
    url = baseURL + '&symbols=' + symbol + '&date_from='+ fromDt + '&date_to='+ toDt
    return requests.get(url).json()

In [4]:
def createDF(resp):
    # Input is the "get" response for a Symbol
    # Return a formatted df with the dates and prices
    dtList = []
    valList = []
    divList = []
    for row in resp['data']:
        dtList.append(row['date'][:10])
        valList.append(row['close'])
        divList.append(row['dividend'])
    d = {}
    d['date'] = dtList
    d['price'] = valList
    d['dividend'] = divList
    return pd.DataFrame(d).sort_values('date')

In [5]:
'''def getStartDt(maxDates, symbol):
    startDt = maxDates.get(symbol, None)
    if startDt:
        startDt =  startDt + datetime.timedelta(days=1)
    else:
        startDt = pd.to_datetime('2012-01-01').date()
    endDt = startDt + datetime.timedelta(days=365)
    return startDt, endDt'''

"def getStartDt(maxDates, symbol):\n    startDt = maxDates.get(symbol, None)\n    if startDt:\n        startDt =  startDt + datetime.timedelta(days=1)\n    else:\n        startDt = pd.to_datetime('2012-01-01').date()\n    endDt = startDt + datetime.timedelta(days=365)\n    return startDt, endDt"

In [17]:
cfg = getConfig()
baseURL = formatURL('eod')

In [18]:
'''df = pd.read_csv(cfg['dataLoc']+'fundData.csv')

df['date'] = df['date'].apply(lambda x: pd.to_datetime(x).date())'''

"df = pd.read_csv(cfg['dataLoc']+'fundData.csv')\n\ndf['date'] = df['date'].apply(lambda x: pd.to_datetime(x).date())"

In [19]:
# The dictionary will hold the most recent date where we have data for each fund
'''currentSymbols = set(df.fund)
maxDates = {}
for s in currentSymbols:
    maxDates[s] = df[df['fund']==s]['date'].max()'''

"currentSymbols = set(df.fund)\nmaxDates = {}\nfor s in currentSymbols:\n    maxDates[s] = df[df['fund']==s]['date'].max()"

In [27]:
symbol = 'GSPC.INDX'    # S&P 500
#symbol = 'URTH'         # MSCI World
symbol = 'F'

In [28]:
today = str(datetime.datetime.today().date())
apiResp = getData(symbol, '2020-01-01', today)
tmp = createDF(apiResp)
tmp

,date,price,dividend
999,2020-02-03,8.98,0.0
998,2020-02-04,9.18,0.0
997,2020-02-05,8.31,0.0
996,2020-02-06,8.25,0.0
995,2020-02-07,8.11,0.0
...,...,...,...
4,2024-01-17,11.27,0.0
3,2024-01-18,10.99,0.0
2,2024-01-19,11.20,0.0
1,2024-01-22,11.20,0.0


In [29]:
tmp['dividend'].sum()

1.8212

In [70]:
fromDt, toDt = getStartDt(maxDates, symbol)
apiResp = getData(symbol, fromDt, toDt)

tmp = createDF(apiResp)
tmp['fund'] = symbol

dfList = []
dfList.append(tmp)
while toDt < datetime.date.today():
    fromDt = toDt + datetime.timedelta(days=1)
    toDt = toDt + datetime.timedelta(days=365)
    apiResp = getData(symbol, fromDt, toDt)
    tmp = createDF(apiResp)
    tmp['fund'] = symbol
    dfList.append(tmp)

upd = pd.concat(dfList).drop_duplicates().dropna()
upd = upd.sort_values(['fund', 'date']).reset_index(drop=True)
upd['date'] = upd['date'].apply(lambda x: pd.to_datetime(x).date())

df = pd.concat([df, upd])

In [71]:
# Check that all funds were updated
df.groupby('fund')['date'].max()

fund
990100.INDX       2022-07-15
ARKK              2022-08-11
CCJ               2022-08-11
COMB              2022-08-11
MSCIALL.INDX      2022-07-15
MSCIEAFE.INDX     2022-07-15
MSCIWORLD.INDX    2022-07-15
QYLD              2022-08-11
RYLD              2022-08-11
SP500TR.INDX      2022-08-12
Name: date, dtype: object

In [58]:
# Just in case I ran the same symbol twice
df = df.drop_duplicates()
# Fill NaN dividend with 0
df.fillna(0, inplace=True)

In [59]:
df.to_csv(cfg['dataLoc']+'fundData.csv', index=False)